In [ ]:
# Import libraries

In [ ]:
import pandas as pd
import json

import re
from nltk.corpus import stopwords

In [ ]:
# Cargar el dataset

In [ ]:
path_data = '../data/Musical_Instruments_5.json'

## Load data

In [ ]:
### load the meta data

data = []
with open(path_data) as f:
    for l in f:
        data.append(json.loads(l.strip()))
    
# total length of list, this number equals total number of products
print(len(data))

# first row of the list
print(data[0])

In [ ]:
# convert list into pandas dataframe

raw_data = pd.DataFrame.from_dict(data)

print(len(raw_data))

In [ ]:
data_reviews = raw_data[['reviewText', 'overall']].copy()
data_reviews.head(3)

In [ ]:
# Chequear por entradas en blanco

In [ ]:
data_reviews.head(3)

In [ ]:
filt = data_reviews['reviewText'] == ''
data_reviews = data_reviews[~filt]

In [ ]:
# Se chequea por entradas duplicadas

dup = data_reviews[data_reviews.duplicated(subset=['reviewText', 'overall'], keep='first')]
dup.shape

In [ ]:
# no hay información en esas 6 filas así que solo las quitaré [esto debería manejarse de otra forma, los duplicados conservar
# el primero y con los valores faltantes eliminarlos]

In [ ]:
data_review_nodup = data_reviews.drop_duplicates(subset=['reviewText', 'overall'],keep=False,inplace=False).copy()
data_review_nodup.shape

In [ ]:
# limpiar los datos
# A list of contractions from http://stackoverflow.com/questions/19790188/expanding-english-language-contractions-in-python

contractions = { 
"ain't": "am not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he's": "he is",
"how'd": "how did",
"how'll": "how will",
"how's": "how is",
"i'd": "i would",
"i'll": "i will",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'll": "it will",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"must've": "must have",
"mustn't": "must not",
"needn't": "need not",
"oughtn't": "ought not",
"shan't": "shall not",
"sha'n't": "shall not",
"she'd": "she would",
"she'll": "she will",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"that'd": "that would",
"that's": "that is",
"there'd": "there had",
"there's": "there is",
"they'd": "they would",
"they'll": "they will",
"they're": "they are",
"they've": "they have",
"wasn't": "was not",
"we'd": "we would",
"we'll": "we will",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"where'd": "where did",
"where's": "where is",
"who'll": "who will",
"who's": "who is",
"won't": "will not",
"wouldn't": "would not",
"you'd": "you would",
"you'll": "you will",
"you're": "you are"
}

In [ ]:
def clean_text(text, remove_stopwords = False):
    '''Remove unwanted characters, stopwords, and format the text to create fewer nulls word embeddings'''
    
    # Convert words to lower case
    text = text.lower()
    
    # Replace contractions with their longer forms 
    if True:
        text = text.split()
        new_text = []
        for word in text:
            if word in contractions:
                new_text.append(contractions[word])
            else:
                new_text.append(word)
        text = " ".join(new_text)
    
    # Format words and remove unwanted characters
    text = re.sub(r'https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
    text = re.sub(r'\<a href', ' ', text)
    text = re.sub(r'&amp;', '', text) 
    text = re.sub(r'[_"\-;%()|+&=*%.,!?:#$@\[\]/]', ' ', text)
    text = re.sub(r'<br />', ' ', text)
    text = re.sub(r'\'', ' ', text)
    text = re.sub(r'[0-9]', '', text )
    text = re.sub('\s+',' ',text) # more than one blank space
    text = text.strip()
    
    # Optionally, remove stop words
    if remove_stopwords:
        text = text.split()
        stops = set(stopwords.words("english"))
        text = [w for w in text if not w in stops]
        text = " ".join(text)

    return text

In [ ]:
# Clean the summaries and texts

clean_texts = []
for text in data_review_nodup.reviewText:
    clean_texts.append(clean_text(text))
print("Texts are complete.")

In [ ]:
# Inspect the cleaned summaries and texts to ensure they have been cleaned well

for i in range(5):
    print("Clean Review #",i+1)
    print(clean_texts[i])
    print()

In [ ]:
clean_texts

In [ ]:
data_reviews.head()

In [ ]:
data_review_nodup['cleaned_reviews'] = clean_texts

In [ ]:
data_review_nodup.head()

In [ ]:
cleaned_reviews = data_review_nodup[['cleaned_reviews', 'overall']]

In [ ]:
cleaned_reviews.loc[0,'cleaned_reviews']

In [ ]:
cleaned_reviews.to_csv('cleaned_reviews.csv', index=False)